In [1]:
%pip install ultralytics 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.yolo as fouy
from ultralytics import YOLO
import random
import os

fo.types.YOLOv5Dataset

fiftyone.types.dataset_types.YOLOv5Dataset

In [9]:
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",
    label_types=["segmentations"],
    classes=["Box"]
)

sample = dataset.first()
sample

Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'open-images-v6-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


<Sample: {
    'id': '696798668eedb29fcff28c5d',
    'media_type': 'image',
    'filepath': 'C:\\Users\\raian\\fiftyone\\open-images-v6\\train\\data\\0000333f08ced1cd.jpg',
    'tags': ['train'],
    'metadata': None,
    'created_at': datetime.datetime(2026, 1, 14, 13, 21, 42, 711000),
    'last_modified_at': datetime.datetime(2026, 1, 14, 13, 21, 42, 711000),
    'ground_truth': <Detections: {

        'detections': [

            <Detection: {


                'id': '696798668eedb29fcff28c5c',


                'attributes': {},


                'tags': [],


                'label': 'Box',


                'bounding_box': [


                    0.153206,


                    0.06875,


                    0.6977519999999999,


                    0.8825000000000001,


                ],


                'mask': array([[False, False, False, ..., False, False, False],


                       [False, False, False, ..., False, False, False],


                       [False, Fals

In [ ]:
session = fo.launch_app(dataset)
session.wait()

In [11]:
export_dir = "./yolo_dataset"

classes = dataset.distinct("ground_truth.detections.label")
print(f"Classes: {classes}")

sample_ids = list(dataset.values("id"))
random.shuffle(sample_ids)

split_idx = int(len(sample_ids) * 0.9)
train_ids = sample_ids[:split_idx]
val_ids = sample_ids[split_idx:]

# Create views
train_view = dataset.select(train_ids)
val_view = dataset.select(val_ids)

print(f"Train samples: {len(train_view)}")
print(f"Val samples: {len(val_view)}")

print("Exporting train split")
train_view.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
    classes=classes,
    split="train",
    use_masks=True
)

print("Exporting val split")
val_view.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
    classes=classes,
    split="val",
    use_masks=True
)

print(f"✓ Export complete to: {os.path.abspath(export_dir)}")

# Verify structure
for root, dirs, files in os.walk(export_dir):
    level = root.replace(export_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')

Classes: ['Adhesive tape', 'Apple', 'Ball', 'Balloon', 'Beer', 'Bicycle wheel', 'Bird', 'Book', 'Boot', 'Bottle', 'Bowl', 'Box', 'Boy', 'Briefcase', 'Cake', 'Camera', 'Car', 'Carnivore', 'Cat', 'Chest of drawers', 'Christmas tree', 'Clothing', 'Coffee', 'Coffee cup', 'Computer keyboard', 'Couch', 'Cowboy hat', 'Dog', 'Dress', 'Drink', 'Facial tissue holder', 'Fedora', 'Filing cabinet', 'Flower', 'Flowerpot', 'Girl', 'Grapefruit', 'Guitar', 'Hat', 'High heels', 'Human body', 'Human ear', 'Human mouth', 'Ipod', 'Jeans', 'Kettle', 'Laptop', 'Lemon', 'Lipstick', 'Loveseat', 'Luggage and bags', 'Man', 'Mobile phone', 'Mouse', 'Muffin', 'Mug', 'Pastry', 'Peach', 'Pen', 'Pencil case', 'Person', 'Picture frame', 'Pillow', 'Pizza', 'Plastic bag', 'Platter', 'Printer', 'Rabbit', 'Rose', 'Scarf', 'Scissors', 'Sculpture', 'Shirt', 'Shorts', 'Sofa bed', 'Sombrero', 'Spoon', 'Strawberry', 'Studio couch', 'Suit', 'Suitcase', 'Sun hat', 'Tablet computer', 'Tap', 'Tart', 'Taxi', 'Teapot', 'Teddy bear',

In [18]:
model = YOLO('yolo11s-seg.pt')

results = model.train(
    data=f"./yolo_dataset/dataset.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name='box_segmentation'
)
print(results)

New https://pypi.org/project/ultralytics/8.4.2 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.228  Python-3.13.5 torch-2.9.1+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=box_segmentation, nbs=64, nm

KeyboardInterrupt: 

In [ ]:
model = YOLO('runs/box_segmentation/weights/best.pt')

for sample in dataset.select_fields("filepath"):
    results = model.predict(sample.filepath, conf=0.25)
    
    detections = []
    for result in results:
        if result.masks is not None:
            for mask, box, conf, cls in zip(
                result.masks.data, 
                result.boxes.xyxy,
                result.boxes.conf,
                result.boxes.cls
            ):
                detections.append(
                    fo.Detection(
                        label=model.names[int(cls)],
                        confidence=float(conf),
                        bounding_box=...,  # Convert box format
                        mask=mask.cpu().numpy()
                    )
                )
    
    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()